In [1]:
import os
#if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#    raise 'stop'
if not os.path.isdir('d:/flagellar/'):
    deps_path = '/kaggle/usr/lib/flg-packages/'
    !pip install --no-index --find-links {deps_path} --requirement {deps_path}/requirements.txt
    !tar xfvz /kaggle/input/ultralytics-for-offline-install/archive.tar.gz
    !pip install --no-index --find-links=./packages ultralytics
    !rm -rf ./packages

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_unet
import flg_numerics
import flg_model
import flg_yolo
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy

fast_mode = True
if fls.is_submission:
    fast_mode = False

MainProcess


In [3]:
all_data = fls.load_all_train_data()+fls.load_all_extra_data()

In [4]:
np.random.default_rng(seed=0).shuffle(all_data)

# Pick N tomograms with 1 motor and N tomograms with 0 motors as validation set
N=100
n_motors = np.array([len(d.labels) for d in all_data])
inds_zero = np.argwhere(n_motors==0)[:N,0]
inds_one = np.argwhere(n_motors==1)[:N,0]
inds_test = np.concatenate((inds_zero,inds_one))
inds_train = np.setdiff1d(np.arange(len(n_motors)), inds_test)
inds_test.shape, inds_train.shape

train_data = []
for i in inds_train:
    train_data.append(all_data[i])
test_data = []
for i in inds_test:
    test_data.append(all_data[i])
np.random.default_rng(seed=0).shuffle(test_data)
test_data = test_data
if fast_mode:
    train_data = train_data[1:3]
    test_data = test_data[1:3]
len(train_data), len(test_data)

(2, 2)

In [8]:
importlib.reload(flg_yolo)
model = flg_yolo.YOLOModel()
model.use_albumentations = False
model.auto_augment = None
model.n_epochs = 1
model.use_pretrained_weights = False
model.train(train_data, test_data)

CompletedProcess(args=['pip', 'uninstall', '-y', 'albumentations'], returncode=0)
Clearing cupy
Will process approximately 27 slices for training


Processing training motors:   0%|          | 0/3 [00:00<?, ?it/s]

Clearing pytorch
Will process approximately 9 slices for validation


Processing validation motors:   0%|          | 0/1 [00:00<?, ?it/s]


Preprocessing Complete:
- Training data: 2 tomograms, 3 motors, 18 slices
- Validation data: 1 tomograms, 1 motors, 9 slices
- Dataset directory: d:/flagellar/temp//yolo_dataset/
- YAML configuration: d:/flagellar/temp//yolo_dataset/dataset.yaml

Ready for YOLO training!
Starting YOLO training process...
Created new YAML at d:/flagellar/temp/training.yaml
Using YAML file: d:/flagellar/temp/training.yaml
YAML contents:
names:
  0: motor
path: d:/flagellar/temp//yolo_dataset/
train: images/train
val: images/val


Starting YOLO training...
New https://pypi.org/project/ultralytics/8.3.102 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.98  Python-3.10.14 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.yaml, data=d:/flagellar/temp/training.yaml, epochs=1, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=4, project=d:/flagellar/temp//yolo_weights/, nam

...

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))




val: Scanning D:\flagellar\temp\yolo_dataset\labels\val.cache... 9 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9

Plotting labels to d:\flagellar\temp\yolo_weights\motor_detector\labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to d:\flagellar\temp\yolo_weights\motor_detector
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size



 0%|          | 0/2 [00:05<?, ?it/s]

RuntimeError: DataLoader worker (pid(s) 22932, 13192, 24448, 6608) exited unexpectedly